<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Подготовка" data-toc-modified-id="Подготовка-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Подготовка</a></span></li><li><span><a href="#Обучение" data-toc-modified-id="Обучение-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Обучение</a></span></li><li><span><a href="#Выводы" data-toc-modified-id="Выводы-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Выводы</a></span></li><li><span><a href="#Чек-лист-проверки" data-toc-modified-id="Чек-лист-проверки-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Чек-лист проверки</a></span></li></ul></div>

# Проект для «Викишоп»

Интернет-магазин «Викишоп» запускает новый сервис. Теперь пользователи могут редактировать и дополнять описания товаров, как в вики-сообществах. То есть клиенты предлагают свои правки и комментируют изменения других. Магазину нужен инструмент, который будет искать токсичные комментарии и отправлять их на модерацию. 

Обучим модель классифицировать комментарии на позитивные и негативные. В нашем распоряжении набор данных с разметкой о токсичности правок.

Построим модель со значением метрики качества *F1* не меньше 0.75. 

**План по выполнению проекта**

1. Загрузим и подготовим данные.
2. Обучим разные модели. 
3. Сделаем выводы.


**Описание данных**

Данные находятся в файле `toxic_comments.csv`. Столбец *text* в нём содержит текст комментария, а *toxic* — целевой признак.

## Подготовка

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pip install catboost
!pip install pytorch-transformers
!pip install keras
!pip install spacy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
ERROR: Operation cancelled by user


In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from sklearn.ensemble import RandomForestClassifier
#from sklearn.model_selection import cross_val_score
from sklearn.metrics import f1_score, make_scorer,accuracy_score
from sklearn.svm import SVC,LinearSVC

from catboost import Pool, CatBoostClassifier



from tqdm import notebook,tqdm, trange
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfVectorizer,CountVectorizer
from sklearn.dummy import DummyClassifier
import lightgbm
 
nltk.download('wordnet','stopwords','punkt','averaged_perceptron_tagger')
from nltk.corpus import wordnet

import time 
 
import warnings
warnings.filterwarnings('ignore')


from sklearn.base import BaseEstimator, TransformerMixin
from gensim.models import Word2Vec
from sklearn.pipeline import Pipeline

from pytorch_transformers import BertTokenizer, BertConfig,AdamW, BertForSequenceClassification

from IPython.display import clear_output
import matplotlib.pyplot as plt
import io

import torch
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
#from keras.preprocessing.sequence import pad_sequences

from keras.utils import pad_sequences
import spacy


In [ ]:
# загрузим дадасет
try:
    df = pd.read_csv('/Users/gazizovilsur26/Downloads/toxic_comments.csv')
except:
    df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/toxic_comments.csv')



In [ ]:
# выведем на экран первые 10 строк
df.head(10)

,Unnamed: 0,text,toxic
0,0,Explanation\nWhy the edits made under my usern...,0
1,1,D'aww! He matches this background colour I'm s...,0
2,2,"Hey man, I'm really not trying to edit war. It...",0
3,3,"""\nMore\nI can't make any real suggestions on ...",0
4,4,"You, sir, are my hero. Any chance you remember...",0
5,5,"""\n\nCongratulations from me as well, use the ...",0
6,6,COCKSUCKER BEFORE YOU PISS AROUND ON MY WORK,1
7,7,Your vandalism to the Matt Shirvington article...,0
8,8,Sorry if the word 'nonsense' was offensive to ...,0
9,9,alignment on this subject and which are contra...,0


In [ ]:
# удалим столбец Unnamed: 0
df = df.drop('Unnamed: 0', axis=1)

In [ ]:
# выведем общую информацию про датасет
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 159292 entries, 0 to 159291
Data columns (total 2 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   text    159292 non-null  object
 1   toxic   159292 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 2.4+ MB


**Промежуточный вывод:**
1. Пропусков нет
2. С типами данных всё хорошо
3. Столбец Unnamed: 0 удалили

In [ ]:
data = df.copy()

In [ ]:
#sample_size = 20000
#data = data.sample(n=sample_size,random_state=12345).reset_index(drop=True)

In [ ]:
# приведём столбец text книжнему регистру
data['text'] = data['text'].str.lower()

In [ ]:
# избавимся от ненужных символов, напишем для этого функцию
def clear_text(text):
    subbed = [' '.join(re.sub(r'[^a-zA-Z0-9]', ' ', text[i]).split()) for i in range(len(text))]
    return subbed


In [ ]:
%%time
data['text'] = clear_text(data['text'])

CPU times: user 6.75 s, sys: 71.1 ms, total: 6.82 s
Wall time: 7.11 s


In [ ]:
from tqdm.notebook import tqdm
tqdm.pandas()
nlp = spacy.load('en_core_web_sm')

def lemmatize_spacy(text):
    return " ".join([token.lemma_ for token in nlp(text)])


In [ ]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 159292 entries, 0 to 159291
Data columns (total 2 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   text    159292 non-null  object
 1   toxic   159292 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 2.4+ MB


In [ ]:
%%time
data['lemm_spacy'] = data['text'].progress_apply(lemmatize_spacy) 

  0%|          | 0/159292 [00:00<?, ?it/s]

CPU times: user 49min 59s, sys: 22.2 s, total: 50min 21s
Wall time: 52min 13s


In [ ]:
data.head()

,text,toxic,lemm_spacy
0,explanation why the edits made under my userna...,0,explanation why the edit make under my usernam...
1,d aww he matches this background colour i m se...,0,d aww he match this background colour I m seem...
2,hey man i m really not trying to edit war it s...,0,hey man I m really not try to edit war it s ju...
3,more i can t make any real suggestions on impr...,0,more I can t make any real suggestion on impro...
4,you sir are my hero any chance you remember wh...,0,you sir be my hero any chance you remember wha...


In [ ]:
# разделим выборки на обучающую и тестовую
train, test = train_test_split(data, test_size=0.9,
                               random_state=12345)

In [ ]:
train_features = train.drop(['text', 'toxic'], axis=1)
train_target = train['toxic']

test_features = test.drop(['text', 'toxic'], axis=1)
test_target = test['toxic']

In [ ]:
train_features.head()

,lemm_spacy
21597,user marnette yes I try no they wouldn t liste...
120811,decline no bye o
22350,1984 hi I just read your comment here user tal...
138088,why do not you play the holy than thou card va...
145477,the my little pony friendship be magic comic b...


In [ ]:
train_target

21597     0
120811    0
22350     0
138088    0
145477    0
         ..
109993    1
85412     0
133249    0
130333    0
77285     0
Name: toxic, Length: 15929, dtype: int64

In [ ]:
train_features['lemm_spacy'].values.astype('U')

array(['user marnette yes I try no they wouldn t listen they take it to wikipedia talk category for discussion question regard category and make a dozen or so revert without any consensus having be reach which be down right cheeky and tendentious take a look at their block log they have a habit or wp 3rr revert when they can t get their own way ignore consensus be one thing invent it to create a revertable pov be bad faith no matter what the edit involve editor make a controversy where not exist in this category create drama for the sake of pov doesn t make end meet 82 8 252 13',
       'decline no bye o',
       '1984 hi I just read your comment here user talk commodore sloat orwellian and have to tell you I know the feeling well I get the definite impression here at the wiki and in the real world as well that 1984 be here it be begin anytime you want to drop I a note please do I would love to hear from you duck talk',
       ...,
       'agree we really should try to stick to the sub

In [ ]:
# TF-IDF
count_tf_idf = TfidfVectorizer(stop_words = 'english')

In [ ]:
tf_idf_train_features = count_tf_idf.fit_transform(train_features['lemm_spacy'].values.astype('U'))
tf_idf_test_features = count_tf_idf.transform(test_features['lemm_spacy'].values.astype('U'))

In [ ]:
print(tf_idf_train_features.toarray())
print(tf_idf_test_features.toarray())

[[0.        0.        0.        ... 0.        0.        0.       ]
 [0.        0.        0.        ... 0.        0.        0.       ]
 [0.        0.        0.        ... 0.        0.        0.       ]
 ...
 [0.0624222 0.        0.        ... 0.        0.        0.       ]
 [0.        0.        0.        ... 0.        0.        0.       ]
 [0.        0.        0.        ... 0.        0.        0.       ]]
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


In [ ]:
train_target

21597     0
120811    0
22350     0
138088    0
145477    0
         ..
109993    1
85412     0
133249    0
130333    0
77285     0
Name: toxic, Length: 15929, dtype: int64

In [ ]:
%%time
# обучаем логистическую регрессию
param_grid = {'C': [0.1, 0.5, 1],'class_weight':['balanced']}
estimator_lr = LogisticRegression()
model_lr = RandomizedSearchCV(
    estimator=estimator_lr, 
    param_distributions = param_grid, 
    scoring='f1', 
    cv=5, 
    verbose=1)
model_lr.fit(tf_idf_train_features, train_target)
print('Лучший набор гиперпараметров для логистической регрессии:', model_lr.best_params_)
print('F1 мера для логистической регресии:', model_lr.best_score_)


Fitting 2 folds for each of 3 candidates, totalling 6 fits
Лучший набор гиперпараметров для логистической регрессии: {'class_weight': 'balanced', 'C': 1}
F1 мера для логистической регресии: 0.715269237202255
CPU times: user 3.2 s, sys: 3.57 s, total: 6.78 s
Wall time: 3.97 s


На валидации TFIDF логистическая регрессия показала результат f1 0.72